In [127]:
#!pip install xgboost

In [128]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import r2_score,mean_squared_error
import warnings
warnings.filterwarnings("ignore")

In [129]:
data=pd.read_csv("D:\\Data Trained\\data(git-hub)\\XGBoost_Vehicles\\XGBoost_Vehicles\\vehicles_data_students.csv")

In [130]:
data.head()

,Unnamed: 0,id,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,county,state,lat,long
0,55483,7315914053,0,2018.0,ram,promaster 2500,excellent,NaN,gas,44244.0,clean,automatic,NaN,NaN,van,NaN,NaN,ca,32.792800,-116.966500
1,162368,7310885048,13995,2017.0,mazda,cx-3,NaN,4 cylinders,gas,7037.0,rebuilt,automatic,NaN,NaN,SUV,white,NaN,ia,41.207382,-96.023096
2,234393,7308243856,19990,2019.0,mitsubishi,eclipse cross sp,good,NaN,gas,35313.0,clean,other,4wd,NaN,hatchback,white,NaN,nc,35.190000,-80.830000
3,276110,7315817729,0,2019.0,honda,cr-v,NaN,NaN,gas,25626.0,clean,automatic,NaN,NaN,SUV,orange,NaN,ny,40.854573,-74.120219
4,349033,7301620999,42900,2015.0,chevrolet,corvette,excellent,8 cylinders,gas,29000.0,clean,automatic,NaN,NaN,convertible,black,NaN,sc,34.755562,-82.906419


In [131]:
data.drop(columns=['Unnamed: 0','id','lat','long','county'],inplace=True)

In [132]:
data.drop('size',inplace=True,axis=1)

In [133]:
data.drop('title_status',inplace=True,axis=1)

In [134]:
data.shape

(64032, 13)

In [135]:
data.isna().sum()

price               0
year              158
manufacturer     2569
model             802
condition       26097
cylinders       26511
fuel              424
odometer          669
transmission      353
drive           19471
type            13785
paint_color     19505
state               0
dtype: int64

In [136]:
data.shape

(64032, 13)

In [137]:
data=data.dropna()

In [138]:
data.shape

(17491, 13)

In [139]:
data=data.drop_duplicates()

In [140]:
data.shape

(16399, 13)

In [141]:
data.describe()

,price,year,odometer
count,1.639900e+04,16399.000000,1.639900e+04
mean,1.611549e+04,2009.294469,1.139066e+05
std,1.336314e+05,9.782876,2.217820e+05
min,0.000000e+00,1918.000000,0.000000e+00
25%,5.500000e+03,2006.000000,5.751000e+04
50%,1.050000e+04,2011.000000,1.038600e+05
75%,2.159000e+04,2015.000000,1.490760e+05
max,1.700000e+07,2022.000000,1.000000e+07


In [142]:
numeric=['int64','int32','int16','int8','float64','float32','float16']
features=data.columns.values.tolist()
categorial_data=[]
for col in features:
    if data[col].dtype in numeric:
        continue
    categorial_data.append(col)

In [143]:
categorial_data

['manufacturer',
 'model',
 'condition',
 'cylinders',
 'fuel',
 'transmission',
 'drive',
 'type',
 'paint_color',
 'state']

In [144]:
df_dummies=pd.get_dummies(data[categorial_data],drop_first=False)
df_dummies.head()

,manufacturer_acura,manufacturer_alfa-romeo,manufacturer_audi,manufacturer_bmw,manufacturer_buick,manufacturer_cadillac,manufacturer_chevrolet,manufacturer_chrysler,manufacturer_datsun,manufacturer_dodge,...,state_sd,state_tn,state_tx,state_ut,state_va,state_vt,state_wa,state_wi,state_wv,state_wy
5,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [145]:
df_dummies.shape

(16399, 4327)

In [146]:
data=data.join(df_dummies)

In [147]:
data.shape

(16399, 4340)

In [148]:
data.drop(columns=categorial_data,inplace=True)

In [149]:
data.shape

(16399, 4330)

In [150]:
data=data[(data.price>1000)&(data.price<40000)]

In [151]:
data.shape

(14742, 4330)

In [152]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14742 entries, 9 to 64031
Columns: 4330 entries, price to state_wy
dtypes: float64(2), int64(1), uint8(4327)
memory usage: 61.3 MB


In [153]:
X=data.drop("price",axis=1)
Y=data.price

In [154]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=10)

In [155]:
import xgboost as xgb
xgb_=xgb.XGBRegressor()
xgb_.fit(X_train,Y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [156]:
pred=xgb_.predict(X_test)

In [157]:
r2_score(Y_test,pred)

0.8449031654434306

In [158]:
0.8449031654434306

0.8449031654434306